In [5]:
import torch
import json
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import torch.nn as nn
from collections import Counter
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from torch.optim import AdamW
from tqdm.auto import tqdm
from torch.nn import CrossEntropyLoss
from sklearn.metrics import precision_recall_fscore_support
from torch.nn.functional import softmax
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report
from transformers import DataCollatorForTokenClassification
import numpy as np
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
import evaluate
from transformers import TrainerCallback, AdamW, get_cosine_schedule_with_warmup

In [6]:
data_files ='../Data/biloc_tagged_clauses.json'
datasets = load_dataset('json', data_files=data_files, field='data')
test_size=0.15
random_seed=42

datasets = datasets['train'].train_test_split(test_size=test_size, seed=random_seed)
print(datasets)

Old caching folder /home/bashyalb/.cache/huggingface/datasets/json/default-f99c3fcb22cc41b9/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96 for dataset json exists but no data were found. Removing it. 


Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['ner_tags', 'id', 'split_tokens'],
        num_rows: 401
    })
    test: Dataset({
        features: ['ner_tags', 'id', 'split_tokens'],
        num_rows: 71
    })
})


In [7]:
datasets["train"][0]["split_tokens"][:15]

['1',
 'EXHIBIT',
 '10.4',
 'SPONSORSHIP',
 'AGREEMENT',
 '---------------------',
 'This',
 'Sponsorship',
 'Agreement',
 'is',
 'made',
 'between',
 'National',
 'Processing',
 'Company']

In [8]:
datasets["train"][0]["ner_tags"][:15]

[0, 0, 0, 0, 0, 0, 0, 1, 3, 0, 0, 0, 5, 6, 7]

In [9]:
with open('../Data/feature_class_labels.json', 'r') as f:
    label_list = json.load(f)

In [10]:
model_name = "bert-base-uncased" 

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [12]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["split_tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [13]:
tokenized_dataset=datasets.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/401 [00:00<?, ? examples/s]

Map:   0%|          | 0/71 [00:00<?, ? examples/s]

In [14]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [15]:
seqeval = evaluate.load("seqeval")

In [22]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [[id2label[p] for p, l in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    true_labels = [[id2label[l] for p, l in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]

    # Assuming you have a seqeval wrapper or configuration that accepts scheme specification
    results = seqeval.compute(predictions=true_predictions, references=true_labels, zero_division=0)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }
    


In [23]:
with open('../Data/feature_class_labels.json', 'r') as f:
    label_list = json.load(f)

In [27]:
id2label = {id: label for id, label in enumerate(label_list)}

label2id = {label: id for id, label in id2label.items()}

In [28]:
model = AutoModelForTokenClassification.from_pretrained(
    "bert-base-uncased", num_labels=165, id2label=id2label, label2id=label2id
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:

training_args = TrainingArguments(
    output_dir="my_awesome_wnut_model",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=100,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="no",  
    load_best_model_at_end=False,  
    push_to_hub=False,  
    logging_dir="./logs",  
    logging_steps=10,  
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.994327,0.000000,0.000000,0.000000,0.869968
2,2.132200,0.905949,0.000000,0.000000,0.000000,0.869968
3,0.912800,0.847343,0.000000,0.000000,0.000000,0.869968
4,0.868500,0.782075,0.000000,0.000000,0.000000,0.869968
5,0.746300,0.725387,0.030303,0.003601,0.006438,0.869292
6,0.690000,0.680321,0.309045,0.147659,0.199838,0.874940
7,0.677800,0.620243,0.336887,0.189676,0.242704,0.877088
8,0.539800,0.587726,0.351181,0.535414,0.424156,0.870843
9,0.518200,0.529854,0.338710,0.277311,0.304950,0.883572
10,0.407900,0.487421,0.409541,0.546218,0.468107,0.890334


TrainOutput(global_step=900, training_loss=0.12635106283757422, metrics={'train_runtime': 687.6485, 'train_samples_per_second': 58.315, 'train_steps_per_second': 1.309, 'total_flos': 1.0493441091072e+16, 'train_loss': 0.12635106283757422, 'epoch': 100.0})

In [30]:
trainer.evaluate()

{'eval_loss': 0.6690486669540405,
 'eval_precision': 0.5404896421845574,
 'eval_recall': 0.6890756302521008,
 'eval_f1': 0.6058047493403694,
 'eval_accuracy': 0.9117740652346857,
 'eval_runtime': 0.9711,
 'eval_samples_per_second': 73.114,
 'eval_steps_per_second': 2.06,
 'epoch': 100.0}

In [31]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
non_trainable_params = total_params - trainable_params

print(f"Total Parameters: {total_params}")
print(f"Trainable Parameters: {trainable_params}")
print(f"Non-trainable Parameters: {non_trainable_params}")

Total Parameters: 109018533
Trainable Parameters: 109018533
Non-trainable Parameters: 0
